In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pathlib
import numpy as np
from tensorflow.keras.preprocessing import image
from skimage.io import imread
import cv2 as cv
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Flatten,Dropout, Conv2D
from tensorflow.keras import Input
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [3]:
def load_image_paths(container_path):
    image_dir = pathlib.Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]

    descr = "A image classification dataset"
    image_paths = []
    flat_data = []
    target = []
    count = 0
    train_img_path = []
    train_label = []
    for i, direc in enumerate(folders):
        for file in direc.iterdir():
            count += 1
            train_img_path.append(str(file))
            train_label.append(categories[i])

    X = np.array(train_img_path)
    y = np.array(train_label)
    catStringArr = np.array(categories)

    return X, y, catStringArr


In [4]:
X = []
y = []
X, y, labelStringArr = load_image_paths("/home/pascal_steiger/Downloads/data/")
img_df = pd.DataFrame(zip(X, y), columns=["path", "label"])

print(labelStringArr)

img_df.sample(n=10)

['MKD' 'HRV' 'ISL' 'IRL' 'ITA' 'HUN' 'LVA' 'GBR' 'FRA' 'LTU' 'AUT' 'SVN'
 'FIN' 'ESP' 'NOR' 'BGR' 'PRT' 'SVK' 'BEL' 'EST' 'POL' 'CZE' 'NLD' 'CHE'
 'DNK' 'SWE']


,path,label
10251,/home/pascal_steiger/Downloads/data/AUT/img_AU...,AUT
13215,/home/pascal_steiger/Downloads/data/ESP/img_ES...,ESP
19645,/home/pascal_steiger/Downloads/data/EST/img_ES...,EST
25063,/home/pascal_steiger/Downloads/data/DNK/img_DN...,DNK
1176,/home/pascal_steiger/Downloads/data/HRV/img_HR...,HRV
549,/home/pascal_steiger/Downloads/data/MKD/img_MK...,MKD
7887,/home/pascal_steiger/Downloads/data/GBR/img_57...,GBR
18883,/home/pascal_steiger/Downloads/data/BEL/img_50...,BEL
9921,/home/pascal_steiger/Downloads/data/LTU/img_LT...,LTU
19670,/home/pascal_steiger/Downloads/data/EST/img_ES...,EST


In [5]:
train_df, test_df = train_test_split(img_df, shuffle=True, test_size=0.2, random_state=42)


print(train_df.sample(n=5))
print(train_df.shape)
print(test_df.sample(n=5))
print(test_df.shape)


                                                    path label
11351  /home/pascal_steiger/Downloads/data/SVN/img_SV...   SVN
1219   /home/pascal_steiger/Downloads/data/HRV/img_HR...   HRV
10788  /home/pascal_steiger/Downloads/data/AUT/img_AU...   AUT
18708  /home/pascal_steiger/Downloads/data/BEL/img_50...   BEL
18904  /home/pascal_steiger/Downloads/data/BEL/img_50...   BEL
(21234, 2)
                                                    path label
8049   /home/pascal_steiger/Downloads/data/FRA/img_48...   FRA
10923  /home/pascal_steiger/Downloads/data/AUT/img_AU...   AUT
18236  /home/pascal_steiger/Downloads/data/SVK/img_SV...   SVK
14920  /home/pascal_steiger/Downloads/data/NOR/img_NO...   NOR
17682  /home/pascal_steiger/Downloads/data/SVK/img_SV...   SVK
(5309, 2)


In [6]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator()

In [15]:
train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col="path",
        y_col="label",
        target_size=(380, 380),
        batch_size=32,
        class_mode="sparse",
        seed=42
)


validation_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col="path",
        y_col="label",
        target_size=(380, 380),
        batch_size=32,
        class_mode="sparse",
        shuffle=False
)

Found 21234 validated image filenames belonging to 26 classes.
Found 5309 validated image filenames belonging to 26 classes.


In [17]:
with tf.device("/device:GPU:0"):
    from tensorflow.keras.applications import EfficientNetB4
    tl_efnet = EfficientNetB4(include_top=True,weights=None)
    tl_efnet = tf.keras.Model(tl_efnet.input, tl_efnet.layers[-2].output)
    
    for layer in tl_efnet.layers:
        layer.trainable = True
    

In [18]:
tl_efnet.summary()


Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 380, 380, 3) 0                                            
__________________________________________________________________________________________________
rescaling_1 (Rescaling)         (None, 380, 380, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
normalization_1 (Normalization) (None, 380, 380, 3)  7           rescaling_1[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 381, 381, 3)  0           normalization_1[0][0]            
____________________________________________________________________________________________

In [19]:
with tf.device("/device:GPU:0"):

    I = Input(shape=(380, 380, 3))



    x = tl_efnet(I)



    dense = Dense(1024, activation='relu', name='denserelu')(x)
    
    dropout = Dropout(0.1)(dense)

    dense2 = Dense(512, activation='relu', name='denserelu2')(dropout)
    
    dropout2 = Dropout(0.1)(dense2)

    dense3 = Dense(26, activation='softmax', name='my_dense')(dropout2)

    #prediction = Dense(26, activation='softmax', name='prediction')



    #out2 = prediction(x)


    new_model2 = tf.keras.Model(I, dense3)

    new_model2.summary()





Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 380, 380, 3)]     0         
_________________________________________________________________
model_5 (Functional)         (None, 1792)              17673823  
_________________________________________________________________
denserelu (Dense)            (None, 1024)              1836032   
_________________________________________________________________
dropout_6 (Dropout)          (None, 1024)              0         
_________________________________________________________________
denserelu2 (Dense)           (None, 512)               524800    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
my_dense (Dense)             (None, 26)                1333

In [21]:
def unwrap_model(model):
    mdl = model.get_layer('model_5')
    inp = mdl.input
    out = model.get_layer('denserelu')(mdl.output)
    out = model.get_layer('dropout_6')(out)
    out = model.get_layer('denserelu2')(out)
    out = model.get_layer('dropout_7')(out)
    out = model.get_layer('my_dense')(out)
    return tf.keras.Model(inp, out)   
    
new_model2 = unwrap_model(new_model2)

    
new_model2.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 380, 380, 3) 0                                            
__________________________________________________________________________________________________
rescaling_1 (Rescaling)         (None, 380, 380, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
normalization_1 (Normalization) (None, 380, 380, 3)  7           rescaling_1[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 381, 381, 3)  0           normalization_1[0][0]            
____________________________________________________________________________________________

Total params: 20,047,993
Trainable params: 19,922,786
Non-trainable params: 125,207
__________________________________________________________________________________________________


In [22]:
# define callbacks for early stopping

early_stopping = EarlyStopping(
    monitor='val_sparse_categorical_accuracy',
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_sparse_categorical_accuracy',
    factor=0.5,
    patience=7,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
]

In [23]:
with tf.device("/device:GPU:0"):
    new_model2.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

with tf.device("/device:GPU:0"):
    history = new_model2.fit(
          train_generator,
          epochs=200,
          verbose = 1,
          callbacks=callbacks,
          #steps_per_epoch=2000 // 32,
          validation_data=validation_generator)
    new_model2.save('CNN-TL_efficientNet_random.h5')

Epoch 1/200


ResourceExhaustedError:  OOM when allocating tensor with shape[32,672,24,24] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model_7/block4b_dwconv/depthwise (defined at <ipython-input-23-bed366476b29>:7) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_83713]

Function call stack:
train_function


In [ ]:
# plot training and validation metrics

sns.set()
fig = plt.figure(0, (12, 4))

ax = plt.subplot(1, 2, 1)
sns.lineplot(x=history.epoch, y=history.history['sparse_categorical_accuracy'], label='train')
sns.lineplot(x=history.epoch, y=history.history['val_sparse_categorical_accuracy'], label='valid')
plt.title('Accuracy')
plt.tight_layout()

ax = plt.subplot(1, 2, 2)
sns.lineplot(x=history.epoch, y=history.history['loss'], label='train')
sns.lineplot(x=history.epoch, y=history.history['val_loss'], label='valid')
plt.title('Loss')
plt.tight_layout()

plt.savefig('CNN(456,456)-tl_efficientnet_random.png')

plt.show()

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        "/mnt/data/testing3/",
        target_size=(224, 224),
        batch_size=1,
        class_mode='sparse',
        shuffle=False)

res=validmodel2.predict(test_generator, steps=len(test_generator), verbose=1)
y_classes = res.argmax(axis=-1)
print(labelStringArr)
label2 = np.sort(labelStringArr)
print(label2)
print(y_classes)

filenames=test_generator.filenames
label_map = (train_generator.class_indices)
print(label_map)
preds = []
labels = []
for i in range(len(filenames)):
    #print(res[i])
    preds.append(res[i][np.argmax(res[i])])
    labels.append(label2[y_classes[i]])




# Data frame
results=pd.DataFrame({"file":filenames, "class":labels, "pr":preds})

results.to_csv("res2.csv", index = False)

In [41]:
from sklearn.metrics import classification_report, confusion_matrix
num_of_test_samples=5309
batch_size=32
Y_pred = new_model2.predict_generator(validation_generator, num_of_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = sorted(labelStringArr)
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

/home/pascal_steiger/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Confusion Matrix
[[ 99   3   2   3   7   5   3   2   2   3   0   4   3   1   2   2   2   2
    4   4   8   2   3   9  16   4]
 [  4  75   1   4   5   8   4   3   1  13  17   1   1  11   0   6   1   1
    0  16   2   3   4   3   0   3]
 [  0   0 153   1   3   3   1   0   0   2   0   5   4   2   2   1   0   0
   12   0   0   3   4   4   5   0]
 [ 10   4   2 147   1   0   1   0   2   2   2   1   1   1   1   1   0   1
    3   3   1   0   2   2   2   4]
 [ 12   5   2   2 101   6   3   6   3   5   3   1   9   3   1   2   3   5
    2   4   3  17   1   5   7   6]
 [  5   3   1   1   5 116   0   2   0   3   6   2   2   7   0   4   1   1
    0  10   8   4   1   4   3   7]
 [  5   4   4   2   7   0 171   1   1   9   5   6   0   3   3   5   0   4
   10   0   5   4  38   9  11   0]
 [  3   2   0   1   5   4   1 126   0   6   0   3   2   1   0   1   8  12
    1   2   0   5   0   1   3   1]
 [  1   2   1   0   1   2   0   2 160   2   0   0   2   2   3   2   1   3
    1   2   8   1   0   0   1  19]
 [